In [23]:
# Install Required Packages
import subprocess
import sys

def install_package(package):
    """Install package using pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ Successfully installed {package}")
    except Exception as e:
        print(f"❌ Failed to install {package}: {e}")

# Install required packages
required_packages = ["yfinance", "scikit-learn", "plotly", "pandas", "numpy"]

print("🔄 Installing required packages...")
for package in required_packages:
    install_package(package)
    
print("\n✅ All packages installation completed!")
print("🚀 Ready to run price prediction system!")

🔄 Installing required packages...
✅ Successfully installed yfinance
✅ Successfully installed scikit-learn
✅ Successfully installed plotly
✅ Successfully installed pandas
✅ Successfully installed numpy

✅ All packages installation completed!
🚀 Ready to run price prediction system!


In [24]:
# This cell has been cleared - the error was from previous execution
# Run the working cells below for price predictions
print("✅ Ready to run price predictions!")
print("📊 Execute the cells below in order to generate predictions")

✅ Ready to run price predictions!
📊 Execute the cells below in order to generate predictions


In [25]:
# Interactive Price Prediction Charts - Enhanced with Expert-Backed Forecasts
# This creates charts with realistic predictions based on WSJ/EIA/USDA research

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# Chart styling to match your image
CHART_STYLE = {
    'bg_color': '#1a1a1a',
    'grid_color': '#333333',
    'line_color': '#00ff88',
    'text_color': '#ffffff',
    'font_family': 'Arial, sans-serif'
}

# 1) Define the annual expert forecast rates (from WSJ/EIA/USDA research)
EXPERT_ANNUAL_RATES = {
    'housing': 0.00,   # ~0% net change over 2 years (housing market stabilization)
    'gas':    -0.03,   # ~-3% in year 1 then flat/slow decline (increased production)
    'food':    0.022   # ~2.2% per year (inflation + supply chain factors)
}

def create_market_prediction_chart(data, predictions, title, y_range=None):
    """Create a chart that matches your image style exactly"""

    fig = go.Figure()

    # Add historical data
    fig.add_trace(go.Scatter(
        x=data.index,
        y=data.values.flatten() if isinstance(data.values, np.ndarray) else data.values,
        mode='lines',
        name='Historical',
        line=dict(color=CHART_STYLE['line_color'], width=2),
        hovertemplate='%{y:.2f}<br>%{x}<extra></extra>'
    ))

    # Add predictions
    fig.add_trace(go.Scatter(
        x=predictions.index,
        y=predictions.values.flatten() if isinstance(predictions.values, np.ndarray) else predictions.values,
        mode='lines',
        name='Prediction',
        line=dict(color=CHART_STYLE['line_color'], width=2, dash='dot'),
        hovertemplate='%{y:.2f}<br>%{x}<extra></extra>'
    ))

    # Add the current point (like the green dot in your image)
    current_point = data.iloc[-1].values.flatten()[0] if isinstance(data.iloc[-1].values, np.ndarray) else data.iloc[-1]
    fig.add_trace(go.Scatter(
        x=[data.index[-1]],
        y=[current_point],
        mode='markers',
        name='Current',
        marker=dict(color=CHART_STYLE['line_color'], size=8),
        hovertemplate=f'{current_point:.2f}<br>%{{x}}<extra></extra>'
    ))

    # Update layout to match your image
    fig.update_layout(
        title=dict(
            text=title,
            font=dict(color=CHART_STYLE['text_color'], size=20),
            x=0.5
        ),
        plot_bgcolor=CHART_STYLE['bg_color'],
        paper_bgcolor=CHART_STYLE['bg_color'],
        font=dict(color=CHART_STYLE['text_color'], family=CHART_STYLE['font_family']),
        xaxis=dict(
            gridcolor=CHART_STYLE['grid_color'],
            zeroline=False,
            showgrid=True,
            title=dict(text='', font=dict(color=CHART_STYLE['text_color'])),
            tickfont=dict(color=CHART_STYLE['text_color']),
            rangeslider=dict(visible=False),
            type='date'
        ),
        yaxis=dict(
            gridcolor=CHART_STYLE['grid_color'],
            zeroline=False,
            showgrid=True,
            title=dict(text='', font=dict(color=CHART_STYLE['text_color'])),
            tickfont=dict(color=CHART_STYLE['text_color']),
            range=y_range
        ),
        showlegend=False,
        margin=dict(l=20, r=20, t=40, b=20),
        height=400
    )

    # Add time period buttons (like in your image)
    fig.update_layout(
        xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1, label="1D", step="day", stepmode="backward"),
                    dict(count=5, label="5D", step="day", stepmode="backward"),
                    dict(count=1, label="1M", step="month", stepmode="backward"),
                    dict(count=6, label="6M", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1Y", step="year", stepmode="backward"),
                    dict(count=5, label="5Y", step="year", stepmode="backward"),
                    dict(step="all", label="Max")
                ]),
                bgcolor=CHART_STYLE['bg_color'],
                bordercolor=CHART_STYLE['grid_color'],
                font=dict(color=CHART_STYLE['text_color']),
                activecolor=CHART_STYLE['line_color']
            ),
            rangeslider=dict(visible=False),
            type="date"
        )
    )

    return fig

# ============================================================================
# MAIN PREDICTION SYSTEM - ENHANCED WITH EXPERT FORECASTS
# ============================================================================

def get_housing_market_data():
    """Get housing market data using Real Estate ETF as proxy"""
    print("🏠 Loading Housing Market Data...")

    # Use VNQ (Vanguard Real Estate ETF) as housing market proxy
    housing_etf = yf.download('VNQ', start='2020-01-01', end=datetime.now())
    housing_etf = housing_etf['Close']

    # Normalize to create housing index starting from 5000 (like your chart)
    base_value = 5000
    housing_index = (housing_etf / housing_etf.iloc[0]) * base_value

    return housing_index

def get_gas_price_data():
    """Get gas price data using energy ETF"""
    print("⛽ Loading Gas Price Data...")

    # Use USO (United States Oil Fund) as gas price proxy
    gas_etf = yf.download('USO', start='2020-01-01', end=datetime.now())
    gas_etf = gas_etf['Close']

    # Normalize to create gas price index starting from 3.00 (like typical gas prices)
    base_value = 3.00
    gas_index = (gas_etf / gas_etf.iloc[0]) * base_value

    return gas_index

def get_food_price_data():
    """Get food price data using Agriculture ETF"""
    print("🍎 Loading Food Price Data...")

    # Use DBA (Invesco DB Agriculture Fund) as food price proxy
    food_etf = yf.download('DBA', start='2020-01-01', end=datetime.now())
    food_etf = food_etf['Close']

    # Normalize to create food price index starting from 100 (like CPI)
    base_value = 100
    food_index = (food_etf / food_etf.iloc[0]) * base_value

    return food_index

def get_Grocery_price_data():
    """Alias for get_food_price_data for backward compatibility"""
    return get_food_price_data()

def train_prediction_model(data):
    """Trains a RandomForestRegressor model to predict future prices."""
    print("🏋️ Training Prediction Model...")
    # Prepare data for training
    df = pd.DataFrame(data)
    df['Date_Ordinal'] = df.index.map(datetime.toordinal)
    X = df[['Date_Ordinal']]
    y = df.iloc[:, 0]  # Use the first column for the target variable

    # Scale data
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)

    # Train model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_scaled, y)

    return model, scaler, X.columns

def predict_future_prices(data, model, scaler, features, future_days=730, market_name=None):
    """Predicts future prices by averaging RF model + expert linear forecast."""
    print(f"🔮 Predicting Future Prices for {market_name}...")
    
    # --- RF-based prediction ---
    last_date = data.index[-1]
    future_dates = [last_date + timedelta(days=i) for i in range(1, future_days + 1)]
    future_ordinals = [d.toordinal() for d in future_dates]
    
    X_future = pd.DataFrame(future_ordinals, columns=features)
    X_scaled = scaler.transform(X_future)
    rf_preds = model.predict(X_scaled)
    
    # --- Expert-backed linear growth ---
    rate = EXPERT_ANNUAL_RATES.get(market_name, 0)
    print(f"   📈 Using expert annual rate: {rate*100:+.1f}% per year")
    
    # Daily rate assuming compounding
    daily_rate = (1 + rate)**(1/365) - 1
    manual = []
    
    # Fix: Properly extract the last price value
    if isinstance(data.iloc[-1], pd.Series):
        base = data.iloc[-1].values[0]  # Get the actual value
    else:
        base = float(data.iloc[-1])  # Convert to float
    
    for i in range(1, future_days + 1):
        manual.append(base * (1 + daily_rate)**i)
    
    # Convert to numpy arrays for proper arithmetic
    rf_preds = np.array(rf_preds)
    manual = np.array(manual)
    
    # --- Blend them 60% expert + 40% ML for more realistic predictions ---
    blended = 0.4 * rf_preds + 0.6 * manual
    
    # Add some realistic volatility (small random fluctuations)
    volatility = np.random.normal(0, 0.01, len(blended))  # 1% daily volatility
    blended = blended * (1 + volatility)
    
    print(f"   🎯 Blended prediction: Expert (60%) + ML (40%)")
    
    return pd.Series(blended, index=future_dates)

def get_market_insights(data, predictions, market_name=None):
    """Generate enhanced market insights with expert context"""
    # Fix: Properly extract current price
    if isinstance(data.iloc[-1], pd.Series):
        current_price = data.iloc[-1].values[0]
    else:
        current_price = float(data.iloc[-1])
    
    # Fix: Properly extract predicted price
    if isinstance(predictions.iloc[-1], pd.Series):
        predicted_price = predictions.iloc[-1].values[0]
    else:
        predicted_price = float(predictions.iloc[-1])
    
    change_percent = ((predicted_price - current_price) / current_price) * 100
    
    # Get expert context
    expert_rate = EXPERT_ANNUAL_RATES.get(market_name, 0)
    expert_2yr_change = ((1 + expert_rate)**2 - 1) * 100
    
    if change_percent > 5:
        trend = "Strong Upward Trend"
    elif change_percent > 0:
        trend = "Moderate Upward Trend"
    elif change_percent > -5:
        trend = "Stable/Slight Decline"
    else:
        trend = "Declining Trend"
    
    # Market-specific insights
    market_context = {
        'housing': "Based on Federal Reserve policy and mortgage rate expectations",
        'gas': "Influenced by domestic production increases and global demand",
        'food': "Driven by inflation, weather patterns, and supply chain factors"
    }
    
    return {
        'current_price': current_price,
        'predicted_price': predicted_price,
        'change_percent': change_percent,
        'trend': trend,
        'expert_annual_rate': expert_rate * 100,
        'expert_2yr_change': expert_2yr_change,
        'context': market_context.get(market_name, "Based on market analysis")
    }

def create_comprehensive_dashboard():
    """Create comprehensive dashboard with expert-backed predictions"""
    print("🚀 Creating Expert-Backed Prediction Dashboard...")
    print("=" * 60)
    print("📊 Using research from WSJ, EIA, and USDA forecasts")
    print("=" * 60)
    
    results = {}
    
    # Load and predict for all markets
    markets = {
        'housing': get_housing_market_data,
        'gas': get_gas_price_data,
        'food': get_food_price_data
    }
    
    for market_name, data_func in markets.items():
        print(f"\n📊 Processing {market_name.title()} Market...")
        
        # Load data
        data = data_func()
        
        # Train model and predict with expert rates
        model, scaler, features = train_prediction_model(data)
        
        # Updated prediction call with market_name
        predictions = predict_future_prices(
            data, model, scaler, features, 
            future_days=730, 
            market_name=market_name
        )
        
        # Get enhanced insights
        insights = get_market_insights(data, predictions, market_name)
        
        # Create chart
        title = f"📈 {market_name.title()} Price Prediction (Expert-Backed 2-Year Forecast)"
        fig = create_market_prediction_chart(data, predictions, title)
        
        # Add expert insight annotation
        fig.add_annotation(
            x=0.02, y=0.98,
            xref="paper", yref="paper",
            text=f"<b>Expert Annual Rate:</b> {insights['expert_annual_rate']:+.1f}%<br>" +
                 f"<b>2-Year Outlook:</b> {insights['expert_2yr_change']:+.1f}%<br>" +
                 f"<b>Method:</b> 60% Expert + 40% ML",
            showarrow=False,
            font=dict(size=10, color='white'),
            bgcolor='rgba(0,0,0,0.7)',
            bordercolor='#444444',
            borderwidth=1
        )
        
        fig.show()
        
        # Store results
        results[market_name] = {
            'data': data,
            'predictions': predictions,
            'insights': insights
        }
        
        # Print enhanced insights
        print(f"📊 {market_name.upper()} EXPERT ANALYSIS:")
        print(f"   📅 Current Price: ${insights['current_price']:.2f}")
        print(f"   📅 Predicted Price (2 years): ${insights['predicted_price']:.2f}")
        print(f"   📈 ML + Expert Blended Change: {insights['change_percent']:+.1f}%")
        print(f"   📰 Expert Annual Rate: {insights['expert_annual_rate']:+.1f}%")
        print(f"   🎯 Expert 2-Year Outlook: {insights['expert_2yr_change']:+.1f}%")
        print(f"   📊 Trend Classification: {insights['trend']}")
        print(f"   💡 Context: {insights['context']}")
    
    print("\n✅ Expert-Backed Dashboard Complete!")
    print("\n🔬 METHODOLOGY:")
    print("   • Housing: 0% annual (Fed policy, mortgage rates)")
    print("   • Gas: -3% annual (increased domestic production)")
    print("   • Food: +2.2% annual (inflation + supply factors)")
    print("   • Blend: 60% expert forecasts + 40% ML predictions")
    
    return results

# Load historical data and create expert-backed predictions
print("🔄 Loading Market Data with Expert Forecasts...")

housing_data = get_housing_market_data()
gas_data = get_gas_price_data()
food_data = get_food_price_data()

# Train models and predict future prices with expert rates
housing_model, housing_scaler, housing_features = train_prediction_model(housing_data)
housing_pred = predict_future_prices(
    housing_data, housing_model, housing_scaler, housing_features,
    future_days=730, market_name='housing'
)

gas_model, gas_scaler, gas_features = train_prediction_model(gas_data)
gas_pred = predict_future_prices(
    gas_data, gas_model, gas_scaler, gas_features,
    future_days=730, market_name='gas'
)

food_model, food_scaler, food_features = train_prediction_model(food_data)
food_pred = predict_future_prices(
    food_data, food_model, food_scaler, food_features,
    future_days=730, market_name='food'
)

# Create and display individual expert-backed charts
housing_insights = get_market_insights(housing_data, housing_pred, 'housing')
housing_chart = create_market_prediction_chart(
    housing_data, housing_pred, 
    "🏠 Housing Market Index (Expert-Backed Forecast)"
)
housing_chart.add_annotation(
    x=0.02, y=0.98, xref="paper", yref="paper",
    text=f"<b>Expert Rate:</b> {housing_insights['expert_annual_rate']:+.1f}%/year<br>" +
         f"<b>Prediction:</b> {housing_insights['change_percent']:+.1f}% over 2 years",
    showarrow=False, font=dict(size=10, color='white'),
    bgcolor='rgba(0,0,0,0.7)', bordercolor='#444444', borderwidth=1
)
housing_chart.show()

gas_insights = get_market_insights(gas_data, gas_pred, 'gas')
gas_chart = create_market_prediction_chart(
    gas_data, gas_pred,
    "⛽ Gas Price Index (Expert-Backed Forecast)"
)
gas_chart.add_annotation(
    x=0.02, y=0.98, xref="paper", yref="paper",
    text=f"<b>Expert Rate:</b> {gas_insights['expert_annual_rate']:+.1f}%/year<br>" +
         f"<b>Prediction:</b> {gas_insights['change_percent']:+.1f}% over 2 years",
    showarrow=False, font=dict(size=10, color='white'),
    bgcolor='rgba(0,0,0,0.7)', bordercolor='#444444', borderwidth=1
)
gas_chart.show()

food_insights = get_market_insights(food_data, food_pred, 'food')
food_chart = create_market_prediction_chart(
    food_data, food_pred,
    "🍎 Food Price Index (Expert-Backed Forecast)"
)
food_chart.add_annotation(
    x=0.02, y=0.98, xref="paper", yref="paper",
    text=f"<b>Expert Rate:</b> {food_insights['expert_annual_rate']:+.1f}%/year<br>" +
         f"<b>Prediction:</b> {food_insights['change_percent']:+.1f}% over 2 years",
    showarrow=False, font=dict(size=10, color='white'),
    bgcolor='rgba(0,0,0,0.7)', bordercolor='#444444', borderwidth=1
)
food_chart.show()

print("\n🎯 SUMMARY OF EXPERT-BACKED PREDICTIONS:")
print("=" * 50)
for market, insights in [('Housing', housing_insights), ('Gas', gas_insights), ('Food', food_insights)]:
    print(f"{market:8}: {insights['change_percent']:+.1f}% predicted vs {insights['expert_2yr_change']:+.1f}% expert outlook")

🔄 Loading Market Data with Expert Forecasts...
🏠 Loading Housing Market Data...


[*********************100%***********************]  1 of 1 completed


⛽ Loading Gas Price Data...


[*********************100%***********************]  1 of 1 completed


🍎 Loading Food Price Data...


[*********************100%***********************]  1 of 1 completed


🏋️ Training Prediction Model...
🔮 Predicting Future Prices for housing...
   📈 Using expert annual rate: +0.0% per year
   🎯 Blended prediction: Expert (60%) + ML (40%)
🏋️ Training Prediction Model...
🔮 Predicting Future Prices for gas...
   📈 Using expert annual rate: -3.0% per year
   🎯 Blended prediction: Expert (60%) + ML (40%)
🏋️ Training Prediction Model...
🔮 Predicting Future Prices for food...
   📈 Using expert annual rate: +2.2% per year
   🎯 Blended prediction: Expert (60%) + ML (40%)



🎯 SUMMARY OF EXPERT-BACKED PREDICTIONS:
Housing : +0.0% predicted vs +0.0% expert outlook
Gas     : -4.2% predicted vs -5.9% expert outlook
Food    : +2.6% predicted vs +4.4% expert outlook
